In [13]:
import pandas as pd
import numpy as np
import sys
import os
import scipy.stats as stats
from statistics import mean
from enum import Enum
import sys
print(sys.version)
import math

class DB(Enum):
    Mcyt=0
    eBioSignDS1=1
    eBioSignDS2=3
    BiosecurID=4
    BiosecureDS2=5
    EvalDB=6

class Signature:
    def __init__(self, data, filePath):
        self.data = data
        self.filePath = filePath
        pathParts= filePath.split('/')
        self.inputDevice=pathParts[2]
        if "signature" in filePath:
            self.db='EvalDb'
            self.inputDevice='Unknown'
            return
        
        parts=pathParts[4].split('_')
        signerId=parts[0][1:]
        self.db=self.getDatabase(pathParts[2],pathParts[3],int(signerId))
        print(self.db)
        pressureColumn=self.getPressureColumn()
        
        self.x=data[:, 0].tolist()
        self.y=data[:, 1].tolist()
        self.t=data[:, 2].tolist()
        self.pressure=data[:, pressureColumn]
        
        if (self.inputDevice == "Unkown"):
            self.inputDevice=getInputDeviceForEvalDb()
        
        self.correctZeroTime()
        self.correctSameTimestamps()
    
        if self.inputDevice == 'stylus':
            i=0
            while len(self.pressure) > 0 and self.pressure[0] == 0:
                self.x.pop(0)
                self.y.pop(0)
                self.t.pop(0)
                self.pressure.pop(0)

            while (len(self.pressure) > 0 and self.pressure[len(self.pressure)-1] == 0):
                self.x.pop(len(self.pressure)-1)
                self.y.pop(len(self.pressure)-1)
                self.t.pop(len(self.pressure)-1)
                self.pressure.pop(len(self.pressure)-1)
        
        self.dx=difference(self.x)
        self.dy=difference(self.y)
        self.dpressure=difference(self.pressure)
        #calculatePathTangentAngle(self)
        self.calculatePathTangentAngle()
        self.calculatePVM()
        self.calculateLCR()
        self.dpvm=difference(self.pvm)
        self.calculateTotalAccelerationMagnitude()
        #self.pta=math.atan(self.dy/self.dx)
        #self.pvm=math.sqrt(self.dx**2+self.dy*2)
        #self.dx=self.x.diff()
        #self.lcr=math.log(self.pvm/self.pta,10)
        print(self.dx)  
        print(self.dy)
        print(self.dpressure)
        print(self.pta)
        print(self.pvm)
        print(self.lcr)
        print(self.tam)
    
    def calculatePathTangentAngle(self):
        self.pta=[]
        for i in range(0,len(self.dx)):     
            if(self.dx[i]==0):
                self.pta.append(math.atan(self.dy[i]/(self.dx[i] + 0.0000000000000001)))
            else:
                self.pta.append(math.atan(self.dy[i]/self.dx[i]))
            
    def calculatePVM(self):
        self.pvm=[]
        for i in range(0,len(self.dx)):
            self.pvm.append(math.sqrt(self.dx[i]**2+self.dy[i]**2))
            
    def calculateLCR(self):
        self.lcr=[]
        for i in range(0,len(self.pvm)):
            value=0
            if(self.pta[i]==0):
                value=self.pvm[i]/(self.pta[i]+0.0000000000000001)
            else:
                self.lcr.append(math.log(self.pvm[i]/self.pta[i],10))
            if(self.pta[i]==0):
                self.lcr.append(math.log(self.pvm[i]/(self.pta[i]+0.0000000000000001),10))
            else:
                self.lcr.append(math.log(self.pvm[i]/self.pta[i],10))
    
    def calculateTotalAccelerationMagnitude(self):
        self.tam=[]
        for i in range(0,len(self.dpvm)):
            self.pta.append(math.sqrt(self.dpvm[i]**2+(self.pvm[i]**2)*(self.pta[i]**2)))
            
    def getPressureColumn(self):
        if self.db==DB.Mcyt:
            return 5
        elif self.db==DB.BiosecurID:
            return 6
        elif self.db==DB.BiosecureDS2:
            return 6
        elif self.db==DB.eBioSignDS1:
            return 3
        elif self.db==DB.eBioSignDS2:
            return 3
        elif self.db==DB.EvalDB:
            return 3
        
    def getDatabase(self, split, inputDevice, signerId):
        '''
        print(split)
        print(inputDevice)
        print(signerId)
        '''
        if split=='Development':
            if inputDevice=='finger':
                if 1009<=signerId<=1038:
                    return DB.eBioSignDS1
                elif 1039<=signerId<=1084:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            elif inputDevice=="stylus":
                if 1<=signerId<=230: 
                    return DB.Mcyt
                elif 231<=signerId<=498:
                    return DB.BiosecurID
                elif 1009<=signerId<=1038:
                    return DB.eBioSignDS1
                elif 1039<=signerId<=1084:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            else:
                print("Undefined InputDevice for file: {file}")
        elif split=='Evaluation':
            if inputDevice=='finger':
                if 373<=signerId<=407:
                    return DB.eBioSignDS1
                elif 408<=signerId<=442:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            elif inputDevice=="stylus":
                if 1<=signerId<=100:
                    return DB.Mcyt
                elif 101<=signerId<=232:
                    return DB.BiosecurID
                elif 233<=signerId<=372:
                    return DB.BiosecureDS2
                elif 373<=signerId<=407:
                    return DB.eBioSignDS1
                elif 408<=signerId<=442:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            else:
                print("Undefined InputDevice for file: {file}")
        else:
            print("Undefined InputDevice for file: {file}")
            
    def getInputDeviceForEvalDb(self):
        for i in range(0, len(self.pressure), 1):
            if self.pressure[i]!=0:
                return False

        return True
    
    def isAllZero(self):
        for i in range(0, self.data.shape[0], 1):
            if self.data[i,0]!=0:
                return False

        return True
    
    def correctZeroTime(self):
        if self.isAllZero():
            for i in range(0, self.data.shape[0], 1):
                self.data[i,0]=i*10
                
    def correctSameTimestamps(self):
        i=0
        while i < self.data.shape[0]-1:
            if self.data[i,0]==self.data[i+1,0]:
                self.data=np.delete(self.data, i, 0)
            else:
                i += 1

def readFilesInDir(dirPath, allFiles):
    signatureFiles = os.listdir(dirPath)
    for file in signatureFiles:
        dataframe=pd.read_csv(dirPath + '/' + file,sep=' ',skiprows=[0],header=None)
        array=np.array(dataframe)
        signature=Signature(array,dirPath + '/' + file)
        allFiles.append(signature)
        
def difference(dataset, interval=1):
    diff = []
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
        
    return diff

def Average(lst):
    return sum(lst) / len(lst)

def normalizeX(allFiles):
    all=[]
    for item in allFiles:
        all=all+item.x
    
    #print('xAll')
    #print(xAll)
    #average=mean(xAll)
    z = stats.zscore(all)
    #print(average)
    print(z)
    allIndex=0
    
    for item in allFiles:
        for i in range(0,len(item.x),1):
            item.x[i]=z[allIndex]
            allIndex+=1
            
    print('Normalized x:')
    for item in allFiles:
        print(item.x)
    '''
    for item in allFiles:
        for i in range(0,len(item.x),1):
            item.x[i]=(item.x[i]-average)/z
    '''
def normalizeY(allFiles):
    all=[]
    for item in allFiles:
        all=all+item.y
    z = stats.zscore(all)
    print(z)
    
    allIndex=0
    for item in allFiles:
        for i in range(0,len(item.y),1):
            item.y[i]=z[allIndex]
            allIndex+=1
            
    print('Normalized y:')
    for item in allFiles:
        print(item.y)
            
def normalizePressure(allFiles):
    all=[]
    for item in allFiles:
        all=all+item.pressure
    z = stats.zscore(all)
    print(z)
    
    allIndex=0
    for item in allFiles:
        for i in range(0,len(item.pressure),1):
            item.pressure[i]=z[allIndex]
            allIndex+=1
            
    print('Normalized pressure:')
    for item in allFiles:
        print(item.pressure)

def normalizeX2(allFiles):
    for item in allFiles:
        z = stats.zscore(item.x)
        print(z)

        for i in range(0,len(item.x),1):
            item.x[i]=z[i]

    print('Normalized x:')
    for item in allFiles:
        print(item.x)
        
def normalizeY2(allFiles):
    for item in allFiles:
        z = stats.zscore(item.y)
        print(z)

        for i in range(0,len(item.y),1):
            item.y[i]=z[i]

    print('Normalized y:')
    for item in allFiles:
        print(item.y)
        
def normalizePressure2(allFiles):
    for item in allFiles:
        z = stats.zscore(item.pressure)
        print(z)

        for i in range(0,len(item.pressure),1):
            item.pressure[i]=z[i]

    print('Normalized pressure:')
    for item in allFiles:
        print(item.pressure)
            
allFiles=[]
        
#readFilesInDir('C:\DeepSignDB/Evaluation/stylus',allFiles)
#readFilesInDir('C:\DeepSignDB/Evaluation/finger',allFiles)
#readFilesInDir('C:\DeepSignDB/Development/stylus',allFiles)

readFilesInDir('D:/DeepSignSmall/Evaluation/stylus',allFiles)
readFilesInDir('D:/DeepSignSmall/Evaluation/finger',allFiles)
readFilesInDir('D:/DeepSignSmall/Development/stylus',allFiles)
readFilesInDir('D:/DeepSignSmall/Development/finger',allFiles)

'''
for file in allFiles:
    #print(file.data)
    #print(file.filePath)
    #print(file.db)
    #print(file.pressure)
'''
    
print('Allfiles:')
#print(allFiles)
print('Loading data finished')

normalizeX2(allFiles)
normalizeY2(allFiles)
normalizePressure2(allFiles)

3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
DB.Mcyt


ValueError: math domain error